In [ ]:
# Timer utilities
import time
from datetime import datetime, timedelta
from IPython.display import clear_output, display

def start_timer(minutes=25, label='Section'):
    """
    Starts a countdown timer that updates the output area every second.
    The cell will run until time is up.
    """
    end = datetime.now() + timedelta(minutes=minutes)
    while True:
        remaining = end - datetime.now()
        if remaining.total_seconds() <= 0:
            clear_output(wait=True)
            display(f'{label}: Time is up!')
            break
        m, s = divmod(int(remaining.total_seconds()), 60)
        clear_output(wait=True)
        display(f'{label}: {m:02d}:{s:02d} remaining')
        time.sleep(1)

## Section Timing
- A: Architecture & Modeling — 25 minutes
- B: Processing & Performance — 25 minutes
- C: Governance & Security — 15 minutes
- D: SQL — 25 minutes

Tip: If you pause mid-section, re-run the timer with remaining minutes.

## Section A — Architecture & Modeling (3 questions)

A1. Real-Time Fraud Detection Architecture
- Design an end-to-end GCP architecture ingesting clickstream and transaction events (millions of users), scoring in real-time, and providing analyst visibility.
- Requirements: Sub-second detection for high-risk events, replay capability, governance with sensitivity tags, DR across regions.
- Deliverables: Ingestion, processing, storage tiers, serving, governance, reliability.

A2. Batch Lakehouse ETL with SCD2
- Design a nightly ETL that ingests dimension tables with SCD2 and fact tables into curated Parquet in GCS and marts in BigQuery.

A3. Multi-region DR for Data Lake
- Provide DR design for lakehouse ensuring RPO ≤ 15 min, RTO ≤ 1 hour.

Add your answers below each question.

In [ ]:
# Start timer for Section A (25 minutes)
start_timer(25, 'Section A')

### Section A — Solutions (Reference)

A1 Solution Outline:
- Ingestion: Pub/Sub (events), Datastream (CDC).
- Processing: Dataflow with sliding windows, feature joins; dead-letter topic.
- Storage: GCS raw Parquet; BigQuery curated features; Bigtable for hot features.
- Serving: Cloud Run scoring API; Looker Studio for monitoring.
- Governance: Dataplex domains; Data Catalog policy tags; CMEK; VPC-SC.
- Reliability: Multi-region topics; autoscaling; backlog replay from Pub/Sub; runbooks.
- Trade-offs: Streaming cost vs latency; BigQuery vs Bigtable for features.

A2 Solution Outline:
- Landing: Raw Avro/Parquet to GCS; schema registry in Dataplex.
- Transform: Dataproc Spark job computes SCD2 (valid_from, valid_to, is_current).
- Serve: BigQuery marts via MERGE into partitioned tables; materialized views.
- Ops: Compaction job; data quality checks; backfill strategy.

A3 Solution Outline:
- GCS dual-region buckets; Pub/Sub topic replication; Dataflow regional failover.
- BigQuery dataset replication; runbooks; IaC with Terraform; tests.

## Section B — Processing & Performance (4 questions)

B1. Spark Skew Troubleshooting
- A join between a 1TB fact and 10GB dimension is slow and fails with executor OOM. Diagnose and fix.

B2. Small Files Problem
- You inherited millions of 5KB Parquet files in GCS. What’s your compaction plan and why?

B3. BigQuery Slow Query
- Query scans 30TB with filter on created_at but still slow. How to optimize?

B4. ETL Failure Due to Overload
- Daily job failed after traffic spike. Outline systematic approach to prevent recurrence.

Add your answers below each question.

In [ ]:
# Start timer for Section B (25 minutes)
start_timer(25, 'Section B')

### Section B — Solutions (Reference)

B1 Solution:
- Diagnose via Spark UI: Shuffle read skew; single heavy partition.
- Fix: Broadcast dimension; salt skewed keys; enable AQE; adjust spark.sql.shuffle.partitions; pre-aggregate.

B2 Solution:
- Spark job to read partitions and write out ~256MB files via coalesce; set maxRecordsPerFile; schedule compaction; update downstream partition expectations.

B3 Solution:
- Ensure partitioning on created_at; cluster by high-cardinality columns; use partition pruning; materialized views; verify slots/reservations; avoid SELECT *.

B4 Solution:
- Observe metrics; find bottlenecks; scale reservations/cluster; add backpressure in Dataflow; implement autoscaling/retries; compact outputs; add alerts.

## Section C — Governance & Security (3 questions)

C1. Column-Level Security for Salary Data
- Ensure HR sees raw salary; others see masked or aggregated.

C2. PII Tokenization for Joinability
- You must join on email without revealing it.

C3. Perimeter Security
- Prevent data exfiltration outside your project perimeter.

Add your answers below each question.

In [ ]:
# Start timer for Section C (15 minutes)
start_timer(15, 'Section C')

### Section C — Solutions (Reference)

C1 Solution:
- BigQuery policy tags on salary with HR group access; views expose masked columns for non-HR; row access policies by department; audit logs; CMEK.

C2 Solution:
- Deterministic hash with salt (SHA256(salt || LOWER(email))); salt in Secret Manager/KMS; store hashed value for joins; rotate salt carefully; document re-identification policy.

C3 Solution:
- VPC-SC around projects/services; restricted service accounts; egress controls; private endpoints; DLP scans.

## Section D — SQL (5 questions)

Assume BigQuery tables:
- orders(order_id, customer_id, region, order_ts TIMESTAMP, order_amount NUMERIC)
- customers(customer_id, region, signup_ts TIMESTAMP)
- events(event_id, customer_id, event_ts TIMESTAMP, event_type STRING)

D1. Top 3 highest-spending customers per region last month.

D2. Monthly retention: customers with an event both this month and last month.

D3. Rolling 7-day average order amount per customer.

D4. Identify first purchase date and days to second purchase per customer.

D5. Customers with no orders in the last 90 days but with events.

Add your answers below each question.

In [ ]:
# Start timer for Section D (25 minutes)
start_timer(25, 'Section D')

### Section D — Solutions (Reference)

D1 Solution:
```sql
WITH last_month AS (
  SELECT * FROM `project.dataset.orders`
  WHERE order_ts >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)
    AND order_ts < DATE_TRUNC(CURRENT_DATE(), MONTH)
), spend AS (
  SELECT customer_id, region, SUM(order_amount) AS total_spend
  FROM last_month
  GROUP BY customer_id, region
)
SELECT region, customer_id, total_spend
FROM (
  SELECT region, customer_id, total_spend,
         ROW_NUMBER() OVER (PARTITION BY region ORDER BY total_spend DESC) AS rn
  FROM spend
)
WHERE rn <= 3
ORDER BY region, total_spend DESC;
```

D2 Solution:
```sql
WITH this_month AS (
  SELECT DISTINCT customer_id
  FROM `project.dataset.events`
  WHERE event_ts >= DATE_TRUNC(CURRENT_DATE(), MONTH)
    AND event_ts < DATE_ADD(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)
), last_month AS (
  SELECT DISTINCT customer_id
  FROM `project.dataset.events`
  WHERE event_ts >= DATE_SUB(DATE_TRUNC(CURRENT_DATE(), MONTH), INTERVAL 1 MONTH)
    AND event_ts < DATE_TRUNC(CURRENT_DATE(), MONTH)
)
SELECT COUNT(*) AS retained
FROM this_month t
JOIN last_month l USING (customer_id);
```

D3 Solution:
```sql
SELECT customer_id,
       DATE(order_ts) AS dt,
       AVG(order_amount) OVER (
         PARTITION BY customer_id
         ORDER BY DATE(order_ts)
         RANGE BETWEEN INTERVAL 6 DAY PRECEDING AND CURRENT ROW
       ) AS avg7
FROM `project.dataset.orders`;
```

D4 Solution:
```sql
WITH o AS (
  SELECT customer_id, order_ts,
         ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_ts) AS rn
  FROM `project.dataset.orders`
)
SELECT customer_id,
       MIN(CASE WHEN rn = 1 THEN DATE(order_ts) END) AS first_purchase_date,
       DATE_DIFF(
         MIN(CASE WHEN rn = 2 THEN DATE(order_ts) END),
         MIN(CASE WHEN rn = 1 THEN DATE(order_ts) END),
         DAY
       ) AS days_to_second
FROM o
GROUP BY customer_id;
```

D5 Solution:
```sql
WITH active_events AS (
  SELECT DISTINCT customer_id
  FROM `project.dataset.events`
  WHERE event_ts >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 90 DAY)
), recent_orders AS (
  SELECT DISTINCT customer_id
  FROM `project.dataset.orders`
  WHERE order_ts >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 90 DAY)
)
SELECT customer_id
FROM active_events ae
LEFT JOIN recent_orders ro USING (customer_id)
WHERE ro.customer_id IS NULL;
```

## Self-Grade & Review
- Architecture: Zones, formats, streaming + batch, catalog, DR.
- Performance: Partitioning, shuffles, skew fixes, file sizing, BigQuery tuning.
- Security: IAM/ABAC, policy tags, masking, CMEK, VPC-SC, auditing.
- SQL: Windows, CTEs, last-month filters, anti/semi joins, arrays.

Write a brief reflection: What trade-offs did you articulate well? Where do you need polish?